In [1]:
import numpy as np
import random

# oh so pretty!
def float_formatter(x):
    if x < 0.001 and x > 0:
        return "   >0" 
    elif x == 0:
        return "    0" 
    else:
        return "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

A = np.random.rand(500,500)
A = A/A.sum(axis=1)[:,None]
print(A.sum(axis=1))

E = np.random.rand(500, 4)
E = E / E.sum(axis=1)[:,None]
print(E.sum(axis=1))
pi = np.random.rand(500)
pi = pi/pi.sum()
print(pi.sum())

#ACGT
def l2i(l):
    d = {'A':0, 'C':1, 'G':2, 'T':3}
    return d[l]

print(l2i('A'))

[1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.

In [2]:
# define all possible outcomes of all states:
outcomes_dict = {} # ohh gawd this is akward
outcomes = np.empty([20], dtype=object) # help me, im lost
i = 0
letters = 'ACGT'
for letter1 in letters:
    outcomes[i] = letter1
    outcomes_dict[letter1] = i
    i += 1
    for letter2 in letters:
        outcomes[i] = letter1 + letter2# + letter3
        #outcomes_dict[letter1 + letter2 + letter3] = i
        outcomes_dict[letter1 + letter2] = i
        i += 1

print(outcomes_dict)
print(outcomes)
#outcomes_dict = alphabet
#outcomes=np.asarray(('A', 'C', 'G', 'T'))
#ACGT
def l2i(l):
    if type(l) is str:
        if l == "":
            return -1
        else:
            return outcomes_dict[l]
    else:
        return [l2i(l_) for l_ in l]
#i can either be a list of indices, or just an index
def i2l(i):
    if type(i) is int:
        return outcomes[i]
    else:
        return [outcomes[i_] for i_ in i]
def e_prop(E, s):
    result = list()
    for i in range(len(s)):
        if i < 0:
            result.append(-1)
        else:
            result.append(E[i, s[i]])
    return result
    

{'GG': 13, 'CA': 6, 'TT': 19, 'AC': 2, 'CG': 8, 'AT': 4, 'AG': 3, 'GC': 12, 'GA': 11, 'CT': 9, 'GT': 14, 'A': 0, 'T': 15, 'TA': 16, 'AA': 1, 'TG': 18, 'G': 10, 'TC': 17, 'C': 5, 'CC': 7}
['A' 'AA' 'AC' 'AG' 'AT' 'C' 'CA' 'CC' 'CG' 'CT' 'G' 'GA' 'GC' 'GG' 'GT'
 'T' 'TA' 'TC' 'TG' 'TT']


In [4]:
#Here we make some test data, to see if viterbi works.

A = np.zeros((3,3))
E = np.zeros((3,outcomes.shape[0]))
pi = np.zeros(3)
A[0, 1] = 0.2
A[0, 2] = 0.2

A[1, 0] = 0.75
A[1, 2] = 0.25
A[2, 0] = 0.75
A[2, 1] = 0.25

E[0, 1] = 0.333 #AA
E[0, 2] = 0.333 #AC
E[0, 3] = 0.333 #AG
E[0, 4] = 0.333 #AT
E[1, 5] = 0.7 #C
E[1, 10] = 0.2 #G
E[1,15] = 0.1 #T
E[2, 5] = 0.2 #C
E[2, 10] = 0.7 #G
E[2 ,15] = 0.1 #T

pi[:] = 1/3

q = np.ones(3, dtype=np.int32)
q[0] = 2
print(A)
print()
print(E)
print()
print(pi)
print()
print(q)

[[    0 0.200 0.200]
 [0.750     0 0.250]
 [0.750 0.250     0]]

[[    0 0.333 0.333 0.333 0.333     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [    0     0     0     0     0 0.700     0     0     0     0 0.200     0
      0     0     0 0.100     0     0     0     0]
 [    0     0     0     0     0 0.200     0     0     0     0 0.700     0
      0     0     0 0.100     0     0     0     0]]

[0.333 0.333 0.333]

[2 1 1]


In [5]:
s_ = "abcde"
s = np.chararray(len(s_), unicode=True)
s[:] = list(s_)
v = np.ones(3, dtype=np.int32)
v[0] = 3
v[1] = 4
v[2] = 1
print(s[2:3])
print(s)
print(s[0])
print(v)
print(s[v])
v[None]

['c']
['a' 'b' 'c' 'd' 'e']
a
[3 4 1]
['d' 'e' 'b']


array([[3, 4, 1]], dtype=int32)

In [5]:
#Calculates the path with maximum likelihood for string s.
#A is transmissions, E is emissions and p is pi

def viterbi_gen(A, E, p, s):
    result = -1*np.ones([A.shape[0], len(s)])
    result[:, 0] = E[:, alphabet[s[0]]]*p
    for i in range(1, len(s)):
        if i % 100 == 0:
            print("\r" + str(i), end="")
        a =  (A.T * result[:, i-1]).max(axis=1)
        b = a * E[:, alphabet[s[i]]]
        result[:, i] = b
    return result

def viterbi_path(X, A, E, p, s):
    n = X.shape[1]-1
    path = np.zeros(n+1, dtype=np.int32)
    path[n] = np.argmax(X[:, n], axis=0)
    for i in range(n-1, -1, -1):
        p = E[path[i+1], alphabet[s[i+1]]]
        p = p * X[:, i]
        p = p * A[:, path[i+1]]
        path[i] = np.argmax(p)
    return path

def viterbi(A, E, p, s):
    X = viterbi_gen(A, E, p, s)
    return viterbi_path(X, A, E, p, s)

In [6]:
print(viterbi(A, E, pi, "ACGTTG"))

NameError: name 'alphabet' is not defined

In [3]:
def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

genome = ''
annotation = ''
for i in range(1,2):
    genome += read_fasta_file('data/genome%d.fa' % i)['genome%d' % i]
    annotation += read_fasta_file('data/annotation%d.fa' % i)['annotation%d' % i]

print(len(genome), len(annotation))

1852441 1852441


In [4]:
npzfile = np.load('codon_hmm_from_counting.npz')
A = npzfile['A']
E = npzfile['E']
pi = npzfile['pi']

# define all possible outcomes of all states:
outcomes_dict = {} # ohh gawd this is akward
outcomes = np.empty([68], dtype=object) # help me, im lost
i = 0
letters = 'ACGT'
for letter1 in letters:
    outcomes[i] = letter1
    outcomes_dict[letter1] = i
    i += 1
    for letter2 in letters:
        for letter3 in letters:
            outcomes[i] = letter1 + letter2 + letter3
            outcomes_dict[letter1 + letter2 + letter3] = i
            i += 1
            
            
print(outcomes)
print(outcomes_dict)


state_codes=npzfile['state_codes']
state_desc=npzfile['state_desc']

q = 3*np.ones(len(state_desc), dtype=np.int32)
q[0] = 1
print(state_desc)
print(q)

def l2i(l):
    if type(l) is str:
        if l == "" or len(l) == 2:
            return -1
        else:
            return outcomes_dict[l]
    else:
        return [l2i(l_) for l_ in l]
#i can either be a list of indices, or just an index
def i2l(i):
    if type(i) is int:
        return outcomes[i]
    else:
        return [outcomes[i_] for i_ in i]
def e_prop(E, s):
    result = list()
    for i in range(len(s)):
        if i < 0:
            result.append(-1)
        else:
            result.append(E[i, s[i]])
    print(result)
    return result

print(type(outcomes))
print(type(outcomes_dict))

['A' 'AAA' 'AAC' 'AAG' 'AAT' 'ACA' 'ACC' 'ACG' 'ACT' 'AGA' 'AGC' 'AGG'
 'AGT' 'ATA' 'ATC' 'ATG' 'ATT' 'C' 'CAA' 'CAC' 'CAG' 'CAT' 'CCA' 'CCC'
 'CCG' 'CCT' 'CGA' 'CGC' 'CGG' 'CGT' 'CTA' 'CTC' 'CTG' 'CTT' 'G' 'GAA'
 'GAC' 'GAG' 'GAT' 'GCA' 'GCC' 'GCG' 'GCT' 'GGA' 'GGC' 'GGG' 'GGT' 'GTA'
 'GTC' 'GTG' 'GTT' 'T' 'TAA' 'TAC' 'TAG' 'TAT' 'TCA' 'TCC' 'TCG' 'TCT'
 'TGA' 'TGC' 'TGG' 'TGT' 'TTA' 'TTC' 'TTG' 'TTT']
{'GCG': 41, 'TCA': 56, 'AAG': 3, 'GAG': 37, 'TAG': 54, 'CAG': 20, 'TTT': 67, 'CTA': 30, 'ATA': 13, 'AAA': 1, 'CGT': 29, 'TTG': 66, 'ATT': 16, 'GCC': 40, 'CAC': 19, 'GCT': 42, 'ACC': 6, 'A': 0, 'GTG': 49, 'ATG': 15, 'TGG': 62, 'TCT': 59, 'CGG': 28, 'GAA': 35, 'CGA': 26, 'T': 51, 'CAA': 18, 'TGT': 63, 'GTT': 50, 'GTA': 47, 'ACA': 5, 'CTT': 33, 'TCC': 57, 'CGC': 27, 'TCG': 58, 'GGC': 44, 'AAT': 4, 'CAT': 21, 'CCG': 24, 'GGA': 43, 'AGA': 9, 'CCT': 25, 'ACG': 7, 'GTC': 48, 'CTG': 32, 'CCA': 22, 'GGG': 45, 'TAT': 55, 'GAC': 36, 'TTC': 65, 'GAT': 38, 'CCC': 23, 'ATC': 14, 'ACT': 8, 'TTA': 64, 

In [24]:
#This is the Viterbi algorithm, where different states, can take input of different length.
#A, E, p and s is as above. q is a vector, signifying the length of inputs for each state.

s = genome

def viterbi_ext_gen(A, E, p, s, q):
    end = np.zeros(A.shape[0], dtype=np.int32)
    A = np.log(A)
    E = (np.log(E).T*((17-5*q)//2)).T
    p = np.log(p)
    #We convert the input string to a numpy char-array
    s_ = np.chararray(len(s), unicode=True)
    s_[:] = list(s)
    s = s_
    #Then we generate some result matrix, with -1 everywhere.
    result = np.log(np.zeros([A.shape[0], len(s)]))
    #We save how long we have gotten in each "row"
    step = np.zeros(A.shape[0], dtype=np.int32)
    #We store the first row, to know the probability of starting in each state.
    s_ = l2i(["".join(s[0: q_]) for q_ in q])
    result[:, 0] = (e_prop(E, s_)+p.T)
    step = step+q
    for i in range(1, len(s)):
        if i % 10000 == 0:
            print("\r" + str(i), end="")
        #First we calculate the argument of the ealier state max probability.
        arg =  (A.T + result[:, i-1]).argmax(axis=1)
        #Then we calculate the actual max
        m = (A.T + result[:, i-1]).max(axis=1)
        #Then we calculate the emission probability
        step_ = step[arg]
        s_ = ["".join(s[step_[i]:step_[i]+q[i]]) for i in range(len(step))]
        s_ = l2i(s_)
        e = np.log(np.zeros(len(s_)))
        for j in range(len(s_)):
            if (s_[j] != -1):
                try:
                    e[j] = E[j, s_[j]]
                except:
                    print("An error happened")
                    print(j)
                    print(len(s_))
                    print(s_)
        #Then we fill in the actual probabilities
        result[:, i] = m+e
        step = step_ + q
        end[step <= len(s)] = i
        highest = np.argmax(result[:, i])
        if step[highest] >= len(s):
            print("Booop")
            print(highest)
            print(step[highest])
            print(step)
            result = result[:, :i+1]
            break
    return result, end

def viterbi_ext_path(X, A, E, p, s, q):
    A = np.log(A)
    E = np.log(E)
    p = np.log(p)
    n = X.shape[1]-1
    j = len(s)
    path = -1*np.ones(n+1, dtype=np.int32)
    path[n] = np.argmax(X[:, n], axis=0)
    for i in range(n, 0, -1):
        q_ = q[path[i]]
        s_ = s[j-q_:j]
        p = E[path[i], l2i(s_)]
        p = p + X[:, i-1]
        p = p + A[:, path[i]]
        path[i-1] = np.argmax(p)
        j = j-q_
    return path

X, end = viterbi_ext_gen(A, E, pi, s, q)
print(X)
print(end)
path = viterbi_ext_path(X, A, E, pi, s, q)
print(path)

/home/morten/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: RuntimeWarning: divide by zero encountered in log
/home/morten/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: RuntimeWarning: divide by zero encountered in log
/home/morten/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in log
/home/morten/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: RuntimeWarning: divide by zero encountered in log
/home/morten/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:34: RuntimeWarning: divide by zero encountered in log


[-6.5664957592460507, -2.4624337939359418, -4.1562480510071698, -inf, -inf, -3.4039117648248869, -inf]
620000Booop
0
1852441
[1852441 1852441 1852443 1852443 1852443 1852441 1852441]
[[-6.566 -13.136 -23.910 ..., -2638844.177 -2638850.747 -2638809.254]
 [-inf -inf -27.978 ..., -inf -inf -inf]
 [-inf -inf -inf ..., -2638791.067 -2638803.009 -inf]
 ..., 
 [-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]]
[620140 620140 620139 620139 620139 620140 620140]


/home/morten/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/home/morten/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:60: RuntimeWarning: divide by zero encountered in log
/home/morten/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:61: RuntimeWarning: divide by zero encountered in log


[0 0 0 ..., 2 3 0]


In [27]:
(path==4).sum()

1370